In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import cv2

In [ ]:
points = np.load("../sp_data/Full_data/points.npy")
width  = 64
height = 256

In [ ]:
def apply_perspective_transform(frame, points,width,height):
        
        assert len(points)==4,"no of points not equal to 4"

        # Define the destination points for the perspective transform
        
        
        dst_points = np.float32([[0, 0], [width, 0], [width, height], [0, height]])

        # Convert points to numpy array format
        src_points = np.float32(points)

        # Compute the perspective transform matrix
        Mat = cv2.getPerspectiveTransform(src_points, dst_points)

        # Apply the perspective transform to the frame
        warped_image = cv2.warpPerspective(frame, Mat, (width, height))

        warped_image  =cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY)

        return warped_image

def process_input(old_frame,new_frame):
        old = apply_perspective_transform(old_frame,points,width,height)
        new = apply_perspective_transform(new_frame,points,width,height)
        np_input = np.array([[old,new]]) #To create a tensor 1 x 2 x H x W
        input_tensor =torch.tensor(np_input,dtype=torch.float32)/255 
        return input_tensor



In [2]:
#Load the dataset
frame_data = np.load("../sp_data/Full_data/framedata.npy",allow_pickle=True)
labeldata = np.load("../sp_data/Full_data/meanlabels2.npy",allow_pickle=True)
frame_data = np.array(frame_data)
print(frame_data.shape,labeldata.shape)
frame_data_tensor = torch.tensor(frame_data,dtype=torch.float32)/255 #Create tensor and normalize
labels_tensor = torch.tensor(2*labeldata/max(labeldata)-1, dtype=torch.float32) #create tensor and normalize

(10199, 2, 256, 64) (10199,)


In [2]:
import torch
import torch.nn as nn

class SpeedNet2(nn.Module):
    def __init__(self, in_channels, f1,f2,f3,f4,f5):
        super(SpeedNet2, self).__init__()
        
        self.batchnorm1 = nn.BatchNorm3d(f1)
        self.batchnorm2 = nn.BatchNorm3d(f2)
        self.batchnorm3 = nn.BatchNorm3d(f3)
        self.batchnorm4 = nn.BatchNorm2d(f4)
        self.batchnorm5 = nn.BatchNorm2d(f5)
        # First 3D convolution layer
        self.conv3d_1 = nn.Conv3d(in_channels, f1, kernel_size=(1, 5, 5))
        self.pool3d_1 = nn.MaxPool3d(kernel_size=(1, 1, 3), stride=(1, 1, 2))
        
        # Second 3D convolution layer
        self.conv3d_2 = nn.Conv3d(f1, f2, kernel_size=(1, 5, 5))
        self.pool3d_2 = nn.MaxPool3d(kernel_size=(1, 1, 2), stride=(1, 1, 2))
        
        # Third 3D convolution layer
        self.conv3d_3 = nn.Conv3d(f2, f3, kernel_size=(2, 5, 5))
        
        # 2D convolution layer
        self.conv2d_1 = nn.Conv2d(f3, f4, kernel_size=(10, 7))
        self.pool2d_1 = nn.MaxPool2d(kernel_size=(9, 1), stride=(9, 2))

        self.conv2d_2 = nn.Conv2d(f4, f5, kernel_size=(5, 1))
        self.pool2d_2 = nn.MaxPool2d(kernel_size=(5, 1), stride=(5, 1))

        



        self.fc1 = nn.Linear(1024, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32,1)
        
    def deb_forward(self, x):
        # Reshape input tensor to add the single channel dimension
        print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        print("After pool3d_1:", x.shape)
        
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        print("After pool3d_2:", x.shape)
        
        # Third 3D convolution
        x = self.conv3d_3(x)
        print("After conv3d_3:", x.shape)
        
        # Squeeze the third dimension
        x = x.squeeze(2)
        print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        print("After conv2d_2:", x.shape)

        x = self.pool2d_2(x)
        print("After pool2d_2:", x.shape)

        
        # Flatten the data
        x = x.view(x.size(0), -1)
        print("After flatten:", x.shape)

        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))fter conv2d_1: torch.Size([2, 128, 235, 2
        # x = self.fc3(x)
        
        
        return x
    
    def forward(self, x):
        # Reshape input tensor to add the single channel dimension
        #print("Before unsqueeze:", x.shape)
        x = x.unsqueeze(1)
        #print("After unsqueeze:", x.shape)
        
        # First 3D convolution and pooling
        x = self.conv3d_1(x)
        #print("After conv3d_1:", x.shape)
        x = self.pool3d_1(x)
        #print("After pool3d_1:", x.shape)
        x = self.batchnorm1(x)
        
        # Second 3D convolution and pooling
        x = self.conv3d_2(x)
        #print("After conv3d_2:", x.shape)
        x = self.pool3d_2(x)
        #print("After pool3d_2:", x.shape)
        x = self.batchnorm2(x)
        # Third 3D convolution
        x = self.conv3d_3(x)
        #print("After conv3d_3:", x.shape)
        x = self.batchnorm3(x)
        # Squeeze the third dimension
        x = x.squeeze(2)
        #print("After squeeze:", x.shape)
        
        # First 2D convolution and pooling
        x = self.conv2d_1(x)
        #print("After conv2d_1:", x.shape)
        x = self.pool2d_1(x)
        x = self.batchnorm4(x)
        #print("After pool2d_1:", x.shape)

        # Second 2D convolution and pooling
        x = self.conv2d_2(x)
        x = self.pool2d_2(x)
        x = self.batchnorm5(x)
        #print("After conv2d_2:", x.shape)
        
        
        # Flatten the data
        x = x.view(x.size(0), -1)
        #print("After flatten:", x.shape)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return x




model = SpeedNet2(1, 8,16,32,128,256)  # Increased features to 128 

<All keys matched successfully>

In [3]:
torch.save(model,'../sp_data/Full_data/SpeedNet_64_full.pth')